In [1]:
import numpy as np
import data
import torch
import argparse
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

In [2]:
parser = argparse.ArgumentParser()
data.add_args(parser)
args = parser.parse_args(['--dataset', 'nbc', '--nbc_features', 'rhand_vel', '--nbc_subsample', '9', '--nbc_mini'])
train_dset = data.dataset_from_args(args, 'train')
test_dset = data.dataset_from_args(args, 'test')

['1_1a_task1', '1_1a_task2', '1_1a_task3', '1_1a_task4', '1_1a_task5', '1_1a_task6']
['3_1b_task1', '3_1b_task2', '3_1b_task3', '3_1b_task4', '3_1b_task5', '3_1b_task6']


In [ ]:
x = train_dset[0]['steps']
y = train_dset[0]['features'].cpu().numpy()
start = (x > 12400).argmax()
end = (x > 12500).argmax()
print(start, end)

plt.plot(x[start:end], y[start:end,0], label='x')
plt.plot(x[start:end], y[start:end,1], label='y')
plt.plot(x[start:end], y[start:end,2], label='z')
plt.legend()
plt.show()

In [13]:
def get_chunks(dset):
    chunks = []
    steps = []
    for i in range(len(dset)):
        feat = dset[i]['features'].cpu().numpy()
        n = (dset[i]['steps'][1:] == 0).argmax()
        if n == 0:
            n = len(dset[i]['steps'])
        for idx in np.arange(0, n - 10, 10):
            chunks.append(feat[idx:idx+10])
            steps.append(dset[i]['steps'][idx])
    chunks = np.stack(chunks)
    steps = np.array(steps)
    return chunks, steps
train_chunks, train_chunk_steps = get_chunks(train_dset)
test_chunks, test_chunk_steps = get_chunks(test_dset)

In [14]:
train_chunks_flattened = train_chunks.reshape(train_chunks.shape[0], -1)
test_chunks_flattened = test_chunks.reshape(test_chunks.shape[0], -1)
neighbors = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(train_chunks_flattened)
nearest, indices = neighbors.kneighbors(train_chunks_flattened)
for idx in indices:
    print(idx, train_chunk_steps[idx])

[  0 518] [11807. 58758.]
[  1 192] [11897. 29266.]
[  2 301] [11987. 39108.]
[  3 150] [12077. 25402.]
[  4 375] [12167. 45830.]
[ 5 33] [12257. 14777.]
[  6 428] [12347. 50600.]
[  7 171] [12437. 27376.]
[ 8 40] [12527. 15407.]
[  9 246] [12617. 34126.]
[ 10 394] [12707. 47540.]
[11 64] [12797. 17567.]
[12 15] [12887. 13157.]
[ 13 250] [12977. 34486.]
[14 68] [13067. 17927.]
[ 15 496] [13157. 56778.]
[16 45] [13247. 15857.]
[17 64] [13337. 17567.]
[ 18 400] [13427. 48080.]
[ 19 167] [13517. 27016.]
[20 24] [13607. 13967.]
[ 21 267] [13697. 36048.]
[ 22 270] [13787. 36318.]
[ 23 167] [13877. 27016.]
[ 24 168] [13967. 27106.]
[ 25 206] [14057. 30526.]
[26 48] [14147. 16127.]
[ 27 333] [14237. 41988.]
[ 28 120] [14327. 22702.]
[ 29 256] [14417. 35026.]
[ 30 394] [14507. 47540.]
[ 31 359] [14597. 44390.]
[32 48] [14687. 16127.]
[33  5] [14777. 12257.]
[ 34 208] [14867. 30706.]
[35  9] [14957. 12617.]
[ 36 195] [15047. 29536.]
[ 37 441] [15137. 51770.]
[ 38 217] [15227. 31516.]
[39 27] [1